## ***<u>Process</u> : $e^+ e^- \to  q \overline{q}$*** 
---

In [1]:
# ─── Import Core Python Modules ───────────────────────────────────────────────
import sys
from pathlib import Path            # For managing filesystem paths
from datetime import datetime       # For timestamping logs
import time                         # For measuring execution duration
from itertools import islice        # For efficient iteration over large files

# ─── Define Key File Paths ────────────────────────────────────────────────────
makefile_path = Path("/home/soumodip/Documents/pythia8312/examples/Makefile.inc")  # Pythia build info
default_lib = "../lib"  # Default fallback for library path

# Path to LHE file for e⁺e⁻ → jj process
lhe_file_path = Path("/home/soumodip/Python/MSc_Project/Finalized_Project/Training_data_75k_100GeV/LHE_Files/e+e-jj_75k_100GeV.lhe")

# Directory to store output data and logs
data_path = Path("/home/soumodip/Python/MSc_Project/Finalized_Project/Training_data_75k_100GeV/JJ_Pipeline/JJ_Datas")
log_path = data_path / "jj_pipeline.log"  # Log file for JJ pipeline

# ─── Logging Utility ──────────────────────────────────────────────────────────
def log(message):
    """Append a timestamped message to the log file."""
    timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    with open(log_path, "a") as f:
        f.write(f"{timestamp} {message}\n")

# ─── Set Pythia8 Library Path Dynamically ─────────────────────────────────────
if makefile_path.is_file():
    with makefile_path.open() as cfg:
        lib = default_lib
        for line in cfg:
            if line.startswith("PREFIX_LIB="):
                lib = line.split("=", 1)[1].strip()  # Use custom lib path if found
                break
        sys.path.insert(0, lib)
        print(f"Library path added: {lib}\n")
        log(f"Library path added: {lib}")
else:
    # Exit if Makefile is not found
    err_msg = f"Error: File '{makefile_path}' does not exist."
    print(err_msg)
    log(err_msg)
    sys.exit(1)

# ─── Import External Libraries After Library Path Is Set ──────────────────────
import pylhe           # For reading LHE (Les Houches Event) files
import pythia8         # For parton showering and hadronization with Pythia8

# ─── Count Total Number of Events in LHE File ─────────────────────────────────
start_time = time.time()

def no_of_events(file_path):
    """Return total number of events in the LHE file."""
    return sum(1 for _ in islice(pylhe.read_lhe_with_attributes(file_path), None))

total_num_events = no_of_events(lhe_file_path)
end_time = time.time()

# ─── Extract Center-of-Mass Energy from LHE Header ────────────────────────────
e_cm = None
with open(lhe_file_path) as f:
    for line in f:
        if line.strip().startswith("<init>"):
            init_line = next(f)
            init_parts = list(map(float, init_line.split()))
            e_cm = init_parts[2] + init_parts[3]  # Sum of incoming beam energies
            break

# ─── Display and Log LHE File Info ────────────────────────────────────────────
print(f"LHE file: {lhe_file_path.name}")
print(f"Number of events: {total_num_events}")
print(f"Center-of-Mass Energy (E_cm): {e_cm:.1f} GeV")
print(f"Time to count events: {end_time - start_time:.2f} seconds\n")

log(f"LHE file: {lhe_file_path.name}")
log(f"Number of events: {total_num_events}")
log(f"Center-of-Mass Energy (E_cm): {e_cm:.1f} GeV")
log(f"Time to count events: {end_time - start_time:.2f} seconds")

# ─── Import Additional Libraries for Jet Clustering and Data Handling ─────────
import numpy as np                      # For numerical operations
import matplotlib.pyplot as plt         # For plotting
import fastjet as fj                    # For jet clustering with FastJet
from scipy.spatial import KDTree        # For efficient nearest-neighbor search
from tqdm import tqdm                   # For progress bars
import h5py                             # For reading/writing HDF5 files
import json                             # For saving skipped events or metadata
from collections import defaultdict     # For flexible counters and mappings

Library path added: /home/soumodip/Documents/pythia8312/lib

LHE file: e+e-jj_75k_100GeV.lhe
Number of events: 75000
Center-of-Mass Energy (E_cm): 100.0 GeV
Time to count events: 3.85 seconds



In [2]:
def truth_lvl_partons(pythia, top_k=1):
    """
    Extract truth-level partons (quarks and gluons) from a Pythia event.

    Args:
        pythia (pythia8.Pythia): Initialized Pythia object after event generation.
        top_k (int): Number of highest-pT gluons to include, in addition to quarks.

    Returns:
        List[dict]: A list of dictionaries with keys: id, pt, eta, phi, e
    """

    # ─── Allowed Quark PDG IDs (u, d, s, c and their antiparticles) ───────────
    pdg_ids_quarks = {1, 2, 3, 4, -1, -2, -3, -4}

    truth_partons = []  # Will store selected quark partons
    gluons = []         # Will store gluons (status -51/-52)

    # ─── Loop Through All Particles in the Event ──────────────────────────────
    for i in range(pythia.event.size()):
        particle = pythia.event[i]

        # Extract relevant kinematic info
        parton_info = {
            "id": particle.id(),
            "pt": particle.pT(),
            "eta": particle.eta(),
            "phi": particle.phi(),
            "e": particle.e(),
        }

        # Select quarks with status -23 (incoming to hard scattering)
        if particle.status() == -23 and particle.id() in pdg_ids_quarks:
            truth_partons.append(parton_info)

        # Select gluons with specific status codes (-51, -52)
        elif particle.status() in (-51, -52) and particle.id() == 21:
            gluons.append(parton_info)

    # ─── Optionally Add Highest-pT Gluons ─────────────────────────────────────
    if gluons:
        gluons_sorted = sorted(gluons, key=lambda x: x["pt"], reverse=True)
        top_gluons = gluons_sorted[:top_k]
        truth_partons.extend(top_gluons)

    # ─── Optional Warning if No Partons Found ─────────────────────────────────
    if not truth_partons:
        warning_msg = "Warning: No truth-level partons found in this event!"
        print(warning_msg)

    return truth_partons

In [3]:
def cluster_jets(event, r, min_jet_pT, eta_max, verbose=False):
    """
    Cluster final-state particles into jets using the anti-kT algorithm.

    Args:
        event (dict): Dictionary with key 'final_particles', containing 4-momenta and PDG ID.
        r (float): Jet radius parameter for anti-kT clustering.
        min_jet_pT (float): Minimum pT threshold for a jet.
        eta_max (float): Maximum |η| allowed for jets.
        verbose (bool): If True, print diagnostics.

    Returns:
        tuple: (list of jets with constituent info, FastJet ClusterSequence object)
    """

    # ─── Check if Event Has Valid Final-State Particles ───────────────────────
    if "final_particles" not in event or not event["final_particles"]:
        raise ValueError("Event does not contain valid final state particles.")

    # ─── Map of PDG IDs to Electric Charge ────────────────────────────────────
    pdg_charge_map = {
        211: +1, -211: -1,      # Charged pions
        321: +1, -321: -1,      # Charged kaons
        2212: +1, -2212: -1,    # Protons and anti-protons
        11: -1, -11: +1,        # Electrons and positrons
        13: -1, -13: +1,        # Muons and anti-muons
        15: -1, -15: +1         # Tau leptons and antileptons
    }

    # ─── Convert Final-State Particles to FastJet PseudoJets ──────────────────
    pseudojets = []
    for px, py, pz, e, pid in event["final_particles"]:
        pj = fj.PseudoJet(px, py, pz, e)    # Create pseudoparticle
        pj.set_user_index(pid)              # Store PDG ID
        pseudojets.append(pj)

    if not pseudojets:
        if verbose:
            print("No valid pseudojets created.")
        return [], None

    # ─── Define Anti-kT Jet Clustering Algorithm ──────────────────────────────
    jet_def = fj.JetDefinition(fj.antikt_algorithm, r, fj.pt_scheme)
    cluster_sequence = fj.ClusterSequence(pseudojets, jet_def)

    # ─── Get All Inclusive Jets Above pT Threshold ────────────────────────────
    jets = cluster_sequence.inclusive_jets(ptmin=min_jet_pT)

    # ─── Apply η Cut ──────────────────────────────────────────────────────────
    jets_with_cuts = [jet for jet in jets if abs(jet.eta()) < eta_max]

    if verbose:
        print(f"Total jets before eta cut: {len(jets)}")
        print(f"Jets passing eta cut: {len(jets_with_cuts)}")

    # ─── Collect Jet and Constituent Information ──────────────────────────────
    jets_with_constituents = []
    for jet in jets_with_cuts:
        constituents = jet.constituents()

        # Skip jets with fewer than 3 constituents
        if len(constituents) < 3:
            if verbose:
                print(f"Skipping jet with {len(constituents)} constituents (less than 3).")
            continue

        constituent_list = []
        for p in constituents:
            pid = p.user_index()
            charge = pdg_charge_map.get(pid, 0)  # Default to 0 if unknown

            constituent_list.append({
                "e": p.e(),
                "pt": p.pt(),
                "eta": p.eta(),
                "phi": p.phi(),
                "pdg_id": pid,
                "charge": charge
            })

        jets_with_constituents.append({
            "eta": jet.eta(),
            "phi": jet.phi(),
            "pt": jet.pt(),
            "mass": jet.m(),
            "multiplicity": len(constituents),
            "charge_multiplicity": sum(1 for c in constituent_list if c["charge"] != 0),
            "constituents": constituent_list
        })

    # ─── Print Warning if No Jet Passes the Multiplicity Cut ──────────────────
    if not jets_with_constituents and verbose:
        print("No jets passed the constituent multiplicity cut.")

    return jets_with_constituents, cluster_sequence

In [4]:
def match_and_tag_jets_with_kdtree(jets, target_partons, r_threshold, verbose=False):
    """
    Match jets to partons using KDTree in (η, φ) space and tag them as quark, antiquark, or gluon.

    Args:
        jets (list): List of clustered jets with η, φ, pt, etc.
        target_partons (list): List of truth-level partons with η, φ, and PDG ID.
        r_threshold (float): Matching radius threshold in (η, φ) space.
        verbose (bool): Print matching summary if True.

    Returns:
        list: Jets with added 'tag' and 'match_distance' keys.
    """

    # ─── Handle Empty Inputs ──────────────────────────────────────────────────
    if not jets or not target_partons:
        if verbose:
            msg = "No jets provided." if not jets else "No target partons provided."
            print(f"{msg}")
        # Return all jets as unmatched
        return [{
            "tag": "unmatched",
            "pt": jet["pt"],
            "eta": jet["eta"],
            "phi": jet["phi"],
            "mass": jet["mass"],
            "multiplicity": jet["multiplicity"],
            "constituents": jet["constituents"],
            "charge_multiplicity": jet["charge_multiplicity"]
        } for jet in jets]

    # ─── Quark PDG IDs (used to classify quark/antiquark jets) ────────────────
    quark_ids = {1, 2, 3, 4}

    # ─── Prepare Coordinates and KDTree for Matching ──────────────────────────
    jets_sorted = sorted(jets, key=lambda x: x["pt"], reverse=True)  # Highest pT first
    parton_coords = np.array([[p["eta"], p["phi"]] for p in target_partons])

    # Wrap φ to handle angular periodicity
    wrapped_coords = np.vstack([
        parton_coords,
        np.array([parton_coords[:, 0], parton_coords[:, 1] + 2 * np.pi]).T,
        np.array([parton_coords[:, 0], parton_coords[:, 1] - 2 * np.pi]).T,
    ])
    kdtree = KDTree(wrapped_coords)

    used_jets = set()       # Track used jet indices
    used_partons = set()    # Track matched parton indices
    matched_jets = []       # Output list with tags and distances

    # ─── Helper: Create Jet Entry with Matching Info ──────────────────────────
    def create_jet_entry(jet, tag, distance=None):
        return {
            "tag": tag,
            "pt": jet["pt"],
            "eta": jet["eta"],
            "phi": jet["phi"],
            "mass": jet["mass"],
            "multiplicity": jet["multiplicity"],
            "constituents": jet["constituents"],
            "charge_multiplicity": jet["charge_multiplicity"],
            "match_distance": distance
        }

    # ─── Match Leading Two Jets to Quarks Only ────────────────────────────────
    for jet_idx, jet in enumerate(jets_sorted[:2]):
        jet_coords = np.array([jet["eta"], jet["phi"]])
        distance, idx = kdtree.query(jet_coords)
        actual_idx = idx % len(parton_coords)
        matched_parton = target_partons[actual_idx]

        if distance < r_threshold and actual_idx not in used_partons:
            pid = matched_parton["id"]
            if abs(pid) in quark_ids:
                tag = "quark" if pid > 0 else "antiquark"
                used_partons.add(actual_idx)
                used_jets.add(jet_idx)
            else:
                tag = "unmatched"
        else:
            tag = "unmatched"

        matched_jets.append(create_jet_entry(jet, tag, distance))

    # ─── Match Remaining Jets to Gluons ───────────────────────────────────────
    for jet_idx, jet in enumerate(jets_sorted[2:], start=2):
        if jet_idx in used_jets:
            continue

        jet_coords = np.array([jet["eta"], jet["phi"]])
        distance, idx = kdtree.query(jet_coords)
        actual_idx = idx % len(parton_coords)
        matched_parton = target_partons[actual_idx]

        if distance < r_threshold and actual_idx not in used_partons:
            if matched_parton["id"] == 21:
                tag = "gluon"
                used_partons.add(actual_idx)
            else:
                tag = "unmatched"
        else:
            tag = "unmatched"

        matched_jets.append(create_jet_entry(jet, tag, distance))

    # ─── Optional: Print Summary of Matching Results ──────────────────────────
    if verbose:
        total_matched = len([j for j in matched_jets if j["tag"] != "unmatched"])
        total_unmatched = len(jets_sorted) - total_matched
        print(f"Jet Matching Summary:")
        print(f"Total jets:     {len(jets_sorted)}")
        print(f"Total partons:  {len(target_partons)}")
        print(f"Matched jets:   {total_matched} ({100 * total_matched / len(jets_sorted):.1f}%)")
        print(f"Unmatched jets: {total_unmatched}")

    return matched_jets

In [5]:
# ─── Initialize Pythia with LHE Input ─────────────────────────────────────────
pythia = pythia8.Pythia()
pythia.readString("Beams:frameType = 4")                   # Use external LHE file
pythia.readString(f"Beams:LHEF = {lhe_file_path}")         # Set the LHE file path
pythia.readString("HardQCD:all = on")                      # Enable QCD processes
pythia.readString("HadronLevel:all = on")                  # Enable hadronization
pythia.init()                                              # Initialize Pythia

# ─── Jet Clustering Configuration ─────────────────────────────────────────────
jet_radius = 0.4                    # Anti-kT jet radius R
delta_r_threshold = 0.32           # Matching threshold in ΔR (η–φ space)
pT_cut = 15.0                      # Minimum jet transverse momentum
eta_cut = 2.5                      # Jet η acceptance range
verbose = True

# ─── Log Jet Clustering Parameters ────────────────────────────────────────────
log(f"Jet clustering parameters:")
log(f"Radius R               = {jet_radius}")
log(f"ΔR match threshold     = {delta_r_threshold}")
log(f"Jet pT cut             = {pT_cut} GeV")
log(f"Jet η cut              = ±{eta_cut}")

# ─── Prepare Containers for Outputs and Errors ────────────────────────────────
truth_partons_list = []    # Stores extracted truth-level partons per event
jets_list = []             # Stores clustered and tagged jets per event
skipped_events = []        # Logs skipped events with reason

start_time = time.time()

# ─── Event Loop ───────────────────────────────────────────────────────────────
for i_event in tqdm(range(total_num_events), desc="Processing Events", unit="event"):
    if not pythia.next():
        # Log if Pythia fails to generate the event
        skipped_events.append({"event_id": i_event, "reason": "Event generation failed."})
        continue

    try:
        # ─── Extract Final-State Particles ────────────────────────────────────
        final_particles = [
            (p.px(), p.py(), p.pz(), p.e(), p.id())
            for p in pythia.event if p.isFinal()
        ]

        if not final_particles:
            skipped_events.append({"event_id": i_event, "reason": "No final-state particles."})
            continue

        event_data = {"final_particles": final_particles}

        # ─── Cluster Jets Using FastJet ───────────────────────────────────────
        jets, cluster_sequence = cluster_jets(event_data, jet_radius, pT_cut, eta_cut)

        if not jets:
            skipped_events.append({"event_id": i_event, "reason": "No jets after clustering."})
            continue

        # ─── Extract Truth-Level Partons ──────────────────────────────────────
        truth_partons = truth_lvl_partons(pythia)

        if not truth_partons:
            skipped_events.append({"event_id": i_event, "reason": "No truth-level partons."})
            continue

        # ─── Match Jets to Partons Using KDTree ──────────────────────────────
        tagged_jets = match_and_tag_jets_with_kdtree(
            jets, truth_partons, delta_r_threshold, verbose=False
        )

        # ─── Store Valid Event Output ────────────────────────────────────────
        truth_partons_list.append({
            "event_id": i_event,
            "truth_partons": truth_partons
        })

        jets_list.append({
            "event_id": i_event,
            "jets": tagged_jets
        })

    except Exception as e:
        # Catch and log any unexpected errors
        skipped_events.append({
            "event_id": i_event,
            "reason": f"Error during processing: {str(e)}"
        })
        continue

# ─── Final Stats and Summary ──────────────────────────────────────────────────
pythia.stat()                        # Print Pythia event statistics
end_time = time.time()

processed = len(truth_partons_list)
skipped = len(skipped_events)
duration = end_time - start_time

# ─── Print and Log Summary ────────────────────────────────────────────────────
print(f"\nSummary:")
print(f"  Processed events: {processed}")
print(f"  Skipped events:   {skipped}")
print(f"  Processing time:  {duration:.3f} seconds")

log(f"Jet clustering complete.")
log(f"Processed events: {processed}")
log(f"Skipped events:   {skipped}")
log(f"Processing time:  {duration:.3f} seconds")


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.312      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 23 May 2024  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 22 Apr 2025 at 12:07:37    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

Processing Events:   0%|                 | 62/75000 [00:00<02:02, 613.76event/s]


 --------  LHA initialization information  ------------ 

  beam    kind      energy  pdfgrp  pdfset 
     A     -11      50.000       0       0
     B      11      50.000       0       0

  Event weighting strategy = -4

  Processes, with strategy-dependent cross section info 
  number      xsec (pb)      xerr (pb)      xmax (pb) 
       1     7.2363e+02     1.7314e-01     7.2363e+02

 --------  End LHA initialization information  -------- 

 --------  LHA event information and listing  ---------------------------------------------------------------------- 

    process =        1    weight =   7.2363e+02     scale =   1.0000e+02 (GeV) 
                        alpha_em =   7.5468e-03    alpha_strong =   1.1638e-01

    Participating Particles 
    no        id stat     mothers     colours      p_x        p_y        p_z         e          m        tau    spin 
     1       -11   -1     0     0     0     0      0.000      0.000     50.000     50.000      0.000   0.000   1.000
     2   

Processing Events:   1%|▏              | 1078/75000 [00:01<01:51, 660.45event/s]


 Pythia::next(): 1000 events have been generated 


...


 Pythia::next(): 73000 events have been generated 


Processing Events:  99%|█████████████▊| 74102/75000 [01:53<00:01, 621.08event/s]


 Pythia::next(): 74000 events have been generated 


Processing Events: 100%|██████████████| 75000/75000 [01:55<00:00, 652.07event/s]


Summary:
  Processed events: 68056
  Skipped events:   6944
  Processing time:  115.027 seconds

 *-------  PYTHIA Event and Cross Section Statistics  -------------------------------------------------------------*
 |                                                                                                                 |
 | Subprocess                                    Code |            Number of events       |      sigma +- delta    |
 |                                                    |       Tried   Selected   Accepted |     (estimated) (mb)   |
 |                                                    |                                   |                        |
 |-----------------------------------------------------------------------------------------------------------------|
 |                                                    |                                   |                        |
 | Les Houches User Process(es)                  9999 |       75000      75000     

In [6]:
# ─── Initialize Counters and Containers ───────────────────────────────────────
truth_counts = defaultdict(int)                   # Counts of truth-level partons (quark, antiquark, gluon)
matched_counts = defaultdict(int)                 # Counts of matched jets by tag
charged_multiplicity_by_class = defaultdict(list) # Stores charged multiplicities per jet class

total_charged_constituents = 0                    # Total number of charged constituents across all jets
total_jet_count = 0                               # Total number of jets processed

# ─── Loop Over All Events ─────────────────────────────────────────────────────
for truth_data, jet_data in zip(truth_partons_list, jets_list):

    # Count truth-level partons by PDG ID
    for parton in truth_data["truth_partons"]:
        pid = parton["id"]
        if pid in {1, 2, 3, 4}:
            truth_counts["quark"] += 1
        elif pid in {-1, -2, -3, -4}:
            truth_counts["antiquark"] += 1
        elif pid == 21:
            truth_counts["gluon"] += 1

    # Count matched jets and collect charged multiplicities
    for jet in jet_data["jets"]:
        tag = jet["tag"]
        charged_mult = jet.get("charge_multiplicity", 0)

        total_charged_constituents += charged_mult
        total_jet_count += 1

        if tag in {"quark", "antiquark", "gluon"}:
            matched_counts[tag] += 1
            charged_multiplicity_by_class[tag].append(charged_mult)

# ─── Efficiency Calculation Helper ─────────────────────────────────────────────
def compute_efficiency(matched, truth):
    return (matched / truth) * 100 if truth > 0 else 0.0

# ─── Print and Log Jet Matching Efficiency ────────────────────────────────────
print("\n=== Jet Matching Summary ===")
log("\n=== Jet Matching Summary ===")

for tag in ["quark", "antiquark", "gluon"]:
    matched = matched_counts[tag]
    truth = truth_counts[tag]
    efficiency = compute_efficiency(matched, truth)
    summary_line = f"{tag.capitalize():<12}: Matched = {matched:6}, Truth = {truth:6}, Efficiency = {efficiency:6.2f}%"
    print(summary_line)
    log(summary_line)

# ─── Compute Average Charged Multiplicity ─────────────────────────────────────
avg_charged_per_jet = total_charged_constituents / total_jet_count if total_jet_count > 0 else 0

print("\n=== Charged Constituents Summary ===")
log("\n=== Charged Constituents Summary ===")

print(f"Total Jets Processed         : {total_jet_count}")
log(f"Total Jets Processed         : {total_jet_count}")
print(f"Total Charged Constituents   : {total_charged_constituents}")
log(f"Total Charged Constituents   : {total_charged_constituents}")
print(f"Average Charged per Jet      : {avg_charged_per_jet:.2f}")
log(f"Average Charged per Jet      : {avg_charged_per_jet:.2f}")

# ─── Print Per-Class Charged Multiplicity Stats ───────────────────────────────
print("\n=== Charged Multiplicity (per class) ===")
log("\n=== Charged Multiplicity (per class) ===")

for tag in ["quark", "antiquark", "gluon"]:
    mults = charged_multiplicity_by_class[tag]
    if mults:
        avg = sum(mults) / len(mults)
        line = f"{tag.capitalize():<12}: Jets = {len(mults):5}, Avg Charged Multiplicity = {avg:6.2f}"
    else:
        line = f"{tag.capitalize():<12}: No matched jets."
    print(line)
    log(line)


=== Jet Matching Summary ===
Quark       : Matched =  50070, Truth =  68056, Efficiency =  73.57%
Antiquark   : Matched =  50167, Truth =  68056, Efficiency =  73.71%
Gluon       : Matched =   1271, Truth =  68031, Efficiency =   1.87%

=== Charged Constituents Summary ===
Total Jets Processed         : 132752
Total Charged Constituents   : 672635
Average Charged per Jet      : 5.07

=== Charged Multiplicity (per class) ===
Quark       : Jets = 50070, Avg Charged Multiplicity =   5.06
Antiquark   : Jets = 50167, Avg Charged Multiplicity =   5.07
Gluon       : Jets =  1271, Avg Charged Multiplicity =   4.69


In [7]:
jet_save_start = time.time()

# Open (or create) an HDF5 file to store jet data
with h5py.File(data_path / "jet_data_jj.h5", "w") as jet_file:
    
    # ─── Store Global Attributes ───────────────────────────────────────────────
    jet_file.attrs["jet_radius"] = jet_radius                          # Jet clustering radius
    jet_file.attrs["pt_cut"] = pT_cut                                  # Minimum jet pT cut
    jet_file.attrs["eta_cut"] = eta_cut                                # Max pseudorapidity for jets
    jet_file.attrs["delta_r_threshold"] = delta_r_threshold            # ΔR matching threshold for tagging
    jet_file.attrs["description"] = (                                  # Description of the dataset
        "Jet dataset from e+e- → jj. Contains jet-level and constituent-level information."
    )
    jet_file.attrs["constituent_format"] = "E, pT, eta, phi, charge, pdg_id"  # Format for constituent data
    jet_file.attrs["n_events"] = len(jets_list)                        # Number of events saved

    # ─── Store Event-wise Jet Data ────────────────────────────────────────────
    for i, jet_data in enumerate(jets_list):
        event_group = jet_file.create_group(f"event_{i}")              # Create a group per event
        jets = jet_data["jets"]

        # ─── Save Jet-Level Metadata ─────────────────────────────────────────
        pt = np.array([jet["pt"] for jet in jets], dtype=np.float32)
        eta = np.array([jet["eta"] for jet in jets], dtype=np.float32)
        phi = np.array([jet["phi"] for jet in jets], dtype=np.float32)
        mass = np.array([jet["mass"] for jet in jets], dtype=np.float32)
        multiplicity = np.array([jet["multiplicity"] for jet in jets], dtype=np.int32)
        n_charged = np.array([jet["charge_multiplicity"] for jet in jets], dtype=np.int32)
        tags = np.array([jet["tag"].encode('utf-8') for jet in jets])  # Encode tags as bytes for HDF5

        # ─── Write Jet-Level Datasets ────────────────────────────────────────
        event_group.create_dataset("pt", data=pt)
        event_group.create_dataset("eta", data=eta)
        event_group.create_dataset("phi", data=phi)
        event_group.create_dataset("mass", data=mass)
        event_group.create_dataset("multiplicity", data=multiplicity)
        event_group.create_dataset("charge_multiplicity", data=n_charged)
        event_group.create_dataset("tag", data=tags)

        event_group.attrs["n_jets"] = len(jets)                        # Number of jets in this event
        event_group.attrs["constituent_format"] = "E, pT, eta, phi, charge, pdg_id"

        # ─── Save Jet Constituent Data ───────────────────────────────────────
        for j, jet in enumerate(jets):
            constituents = np.array([
                [c["e"], c["pt"], c["eta"], c["phi"], c["charge"], c["pdg_id"]]
                for c in jet["constituents"]
            ], dtype=np.float32)

            # Store with compression to save space
            event_group.create_dataset(
                f"jet_{j}_constituents",
                data=constituents,
                compression="gzip",
                compression_opts=9,
                shuffle=True
            )

jet_save_end = time.time()

# ─── Completion Logs ─────────────────────────────────────────────────────────
print(f"Jet data saved to: {data_path / 'jet_data_jj.h5'}")
print(f"Time taken: {jet_save_end - jet_save_start:.2f} seconds")

Jet data saved to: /home/soumodip/Python/MSc_Project/Finalized_Project/Training_data_75k_100GeV/JJ_Pipeline/JJ_Datas/jet_data_jj.h5
Time taken: 101.98 seconds


In [8]:
truth_save_start = time.time()

# Open (or create) an HDF5 file to store truth-level parton data
with h5py.File(data_path / "truth_parton_data_jj.h5", "w") as truth_file:
    
    # ─── Global Attributes ────────────────────────────────────────────────
    truth_file.attrs["description"] = "Truth-level partons for e+e- → jj"  # File description
    truth_file.attrs["columns"] = "id, pt, eta, phi, e"                    # Format of stored columns
    truth_file.attrs["n_events"] = len(truth_partons_list)                # Number of events stored

    # ─── Event-wise Data Saving ──────────────────────────────────────────
    for i, truth_data in enumerate(truth_partons_list):
        event_group = truth_file.create_group(f"event_{i}")               # Create a group per event
        partons = truth_data["truth_partons"]

        # Extract parton-level data into separate arrays
        parton_ids = np.array([p["id"] for p in partons], dtype=np.int32)
        parton_pt  = np.array([p["pt"] for p in partons], dtype=np.float32)
        parton_eta = np.array([p["eta"] for p in partons], dtype=np.float32)
        parton_phi = np.array([p["phi"] for p in partons], dtype=np.float32)
        parton_e   = np.array([p["e"] for p in partons], dtype=np.float32)

        # ─── Save each attribute with compression ────────────────────────
        event_group.create_dataset("id",   data=parton_ids, compression="gzip", compression_opts=9, shuffle=True)
        event_group.create_dataset("pt",   data=parton_pt,  compression="gzip", compression_opts=9, shuffle=True)
        event_group.create_dataset("eta",  data=parton_eta, compression="gzip", compression_opts=9, shuffle=True)
        event_group.create_dataset("phi",  data=parton_phi, compression="gzip", compression_opts=9, shuffle=True)
        event_group.create_dataset("e",    data=parton_e,   compression="gzip", compression_opts=9, shuffle=True)

        # Store number of partons as group-level metadata
        event_group.attrs["n_partons"] = len(partons)

truth_save_end = time.time()

# ─── Completion Message ──────────────────────────────────────────────────
print(f"Truth parton data saved to: {data_path / 'truth_parton_data_jj.h5'}")
print(f"Time taken: {truth_save_end - truth_save_start:.2f} seconds")

Truth parton data saved to: /home/soumodip/Python/MSc_Project/Finalized_Project/Training_data_75k_100GeV/JJ_Pipeline/JJ_Datas/truth_parton_data_jj.h5
Time taken: 66.02 seconds


In [9]:
skipped_save_start = time.time()

# ─── Prepare a Clean List of Skipped Events ───────────────────────────────
# Extract only event ID and skip reason for each skipped event
skipped_data = [
    {"event_id": s["event_id"], "reason": s["reason"]}
    for s in skipped_events
]

# ─── Save Skipped Events to JSON File ─────────────────────────────────────
# Write the cleaned skipped event list to a JSON file (human-readable with indent=2)
with open(data_path / "skipped_events_data_jj.json", "w") as f:
    json.dump(skipped_data, f, indent=2)

skipped_save_end = time.time()

# ─── Print Completion Message ─────────────────────────────────────────────
print(f"Skipped events data saved to: {data_path / 'skipped_events_data_jj.json'}")
print(f"Skipped events saved in {skipped_save_end - skipped_save_start:.2f} seconds.")

Skipped events data saved to: /home/soumodip/Python/MSc_Project/Finalized_Project/Training_data_75k_100GeV/JJ_Pipeline/JJ_Datas/skipped_events_data_jj.json
Skipped events saved in 0.03 seconds.
